# Widefield tutorial

This tutorial demonstraces how to access the *Widefield dataset* using `pynwb`. 

This dataset contains the Widefield imaging data, behavior measurements from the ViRMEN system and pose estimation data from LightningPose.

Contents:

- [Reading an NWB file](#read-nwb)
- [Access subject and task metadata](#access-subject)
- [Access Imaging](#access-imaging)
- [Access Behavior](#access-behavior)
- [View NWB files](#view-nwb)

A schematic representation where the source data is saved in NWB: 

![Alt text](data_types_eyetracking.png)

# Reading an NWB file <a name="read-nwb"></a>

This section demonstrates how to read an NWB file using `pynwb`.

Based on the [NWB File Basics](https://pynwb.readthedocs.io/en/stable/tutorials/general/plot_file.html#sphx-glr-tutorials-general-plot-file-py) tutorial from [PyNWB](https://pynwb.readthedocs.io/en/stable/#).

An [NWBFile](https://pynwb.readthedocs.io/en/stable/pynwb.file.html#pynwb.file.NWBFile) represents a single session of an experiment. Each NWBFile must have a `session description`, `identifier`, and `session start time`.

Reading is carried out using the [NWBHDF5IO](https://pynwb.readthedocs.io/en/stable/pynwb.html#pynwb.NWBHDF5IO) class. To read the NWB file use the read mode ("r") to retrieve an NWBFile object.


In [ ]:
from pynwb import NWBHDF5IO

# The file path to a .nwb file
nwbfile_path = "/Users/weian/data/210944/sub-Cherry/sub-Cherry_ses-20230802-20hz-1_behavior+image+ophys.nwb"
io = NWBHDF5IO(path=nwbfile_path, mode="r", load_namespaces=True)
nwbfile = io.read()

nwbfile

Importantly, the `session start time` is the reference time for all timestamps in the file. For instance, an event with a timestamp of 0 in the file means the event occurred exactly at the session start time.

The `session_start_time` is extracted from the `date` and `time` variables from the ViRMEN file.

In [ ]:
nwbfile.session_start_time

# Access subject and task related metadata <a name="access-subject"></a>

## Access subject metadata

This section demonstrates how to access the [Subject](https://pynwb.readthedocs.io/en/stable/pynwb.file.html#pynwb.file.Subject) field in an NWB file.

The [Subject](https://pynwb.readthedocs.io/en/stable/pynwb.file.html#pynwb.file.Subject) field can be accessed as `nwbfile.subject`.


In [ ]:
nwbfile.subject

## Access ViRMEN experimental metadata (mazes table, stimulus protocol parameters)

This section demonstrates how to access the task related metadata in an NWB file.

The ViRMEN experimental metadata is stored in a [LabMetaData](https://pynwb.readthedocs.io/en/stable/pynwb.file.html#pynwb.file.LabMetaData) extension ([ndx-pinto-metadata](https://github.com/catalystneuro/ndx-pinto-metadata)). 

The [LabMetaData](https://pynwb.readthedocs.io/en/stable/pynwb.file.html#pynwb.file.LabMetaData) object can be accessed as `nwbfile.lab_meta_data["LabMetaData"]`.

The parameters for the mazes is added to the `mazes` table within `nwbfile.lab_meta_data["LabMetaData"]` which can be accessed as `nwbfile.lab_meta_data["LabMetaData"].mazes`. 

Data arrays are read passively from the file. Accessing the data attribute of the `mazes` object does not read the data values, but presents an HDF5 object that can be indexed to read data. You can use the [:] operator to read the entire data array into memory as `nwbfile.lab_meta_data["LabMetaData"].mazes[:]`.

In [ ]:
nwbfile.lab_meta_data["LabMetaData"]

In [ ]:
nwbfile.lab_meta_data["LabMetaData"].mazes[:]

In [ ]:
nwbfile.lab_meta_data["LabMetaData"].stimulus_protocol[:]

## Access trials

Behavior trials are stored in `nwbfile.trials`. The `start_time` denotes the start time of each trial in seconds relative to the global session start time (using the "StartOfTrial" column from ViRMEN `.mat` file).
The `stop_time` denotes the end time of each trial in seconds relative to the global session start time
(using the "EndOfTrial" column from the ViRMEN `.mat` file).

`nwbfile.trials` can be converted to a pandas DataFrame for convenient analysis using `nwbfile.trials.to_dataframe`.


In [ ]:
trials = nwbfile.trials.to_dataframe()

trials[:10]

In [ ]:
trials[trials["trial_type"] == "right"][:3]

# Access Widefield Imaging <a name="access-imaging"></a>

This section demonstraces how to access the raw and processed Widefield imaging data.

`NWB` organizes data into different groups depending on the type of data. Groups can be thought of as folders within the file. Here are some of the groups within an NWBFile and the types of data they are intended to store:

- `acquisition`: raw, acquired data that should never change
- `processing`: processed data, typically the results of preprocessing algorithms and could change

## Raw Widefield Imaging

The raw Widefield imaging data is stored in `pynwb.ophys.OnePhotonSeries` objects (blue and violet separately) which is added to `nwbfile.acquisition`. The blue frames can be accessed as `nwbfile.acquisition['OnePhotonSeriesBlue']`, the violet frames as `nwbfile.acquisition['OnePhotonSeriesViolet']`.

The data in [OnePhotonSeries](https://pynwb.readthedocs.io/en/stable/pynwb.ophys.html#pynwb.ophys.OnePhotonSeries) is stored as a three dimensional array: the first dimension is time (frame), the second and third dimensions represent x and y (width by height). 

In [ ]:
photon_series_blue = nwbfile.acquisition['OnePhotonSeriesBlue']
photon_series_violet = nwbfile.acquisition['OnePhotonSeriesViolet']

In [ ]:
# Visualize the imaging data.

from matplotlib import pyplot as plt

plt.imshow(photon_series_blue.data[50].T, aspect="auto", cmap="Greys")
plt.title("Blue")
plt.show()

plt.imshow(photon_series_violet.data[50].T, aspect="auto", cmap="Greys")
plt.title("Violet")
plt.show()


The timestamps for the blue frames can be accessed as `nwbfile.acquisition['OnePhotonSeriesBlue'].timestamps`.

The blue frame timestamps that are aligned with the behavior clock are added from the `wf_behav_sync.mat` file.
The violet frame timestamps are aligned to the blue frame timestamps by interpolation.

In [ ]:
photon_series_blue.timestamps[:100]

The processed imaging data is stored in the "ophys" processing module, which can be accessed as `nwbfile.processing["ophys"]`.

In [ ]:
nwbfile.processing["ophys"]

## Motion Correction

The x,y shifts for the blue and violet frames is added as [TimeSeries](https://pynwb.readthedocs.io/en/stable/pynwb.base.html#pynwb.base.TimeSeries) objects.

The motion correction series for the blue frames can be accessed as `nwbfile.processing["ophys"]["MotionCorrectionSeriesBlue"]`.

The timestamps for the motion correction series references the same timestamps as for the raw imaging data.

In [ ]:
motion_correction_blue = nwbfile.processing["ophys"]["MotionCorrectionSeriesBlue"]
motion_correction_blue

In [ ]:
motion_correction_blue.data[:10], motion_correction_blue.timestamps[:10]

## Processed Widefield Imaging

The downsampled imaging data for the blue frames can accessed as `nwbfile.processing["ophys"]["OnePhotonSeriesProcessedBlue"]`.

The data in [OnePhotonSeries](https://pynwb.readthedocs.io/en/stable/pynwb.ophys.html#pynwb.ophys.OnePhotonSeries) is stored as a three dimensional array: the first dimension is time (frame), the second and third dimensions represent x and y (width by height). 

The binned image size is (128, 128) which can be accessed with the "dimension" attribute in `nwbfile.processing["ophys"]["OnePhotonSeriesProcessedBlue"]`.

In [ ]:
nwbfile.processing["ophys"]["OnePhotonSeriesProcessedBlue"]

In [ ]:
nwbfile.processing["ophys"]["OnePhotonSeriesProcessedBlue"].dimension[:]

## Accessing the segmentation data

The segmentation output for the Widefield Imaging data is stored in `nwbfile.processing["ophys"]`. 

In NWB, the [PlaneSegmentation](https://pynwb.readthedocs.io/en/stable/pynwb.ophys.html#pynwb.ophys.PlaneSegmentation) class stores the detected regions of interest in the [OnePhotonSeries](https://pynwb.readthedocs.io/en/stable/pynwb.ophys.html#pynwb.ophys.OnePhotonSeries) data. The [ImageSegmentation](https://pynwb.readthedocs.io/en/stable/pynwb.ophys.html#pynwb.ophys.ImageSegmentation) can contain multiple `PlaneSegmentation` tables, so that we can store results of different segmentation algorithms or different segmentation classes.

We can access the plane segmentation for the processed [OnePhotonSeries](https://pynwb.readthedocs.io/en/stable/pynwb.ophys.html#pynwb.ophys.OnePhotonSeries) data (blue frames) as 
`nwbfile.processing["ophys"]["ImageSegmentation"]["PlaneSegmentationProcessedBlue"]`.


In [ ]:
plane_segmentation = nwbfile.processing["ophys"]["ImageSegmentation"]["PlaneSegmentationProcessedBlue"][:]
plane_segmentation[:10]

The summary images of the segmentation are stored in [Images](https://pynwb.readthedocs.io/en/stable/pynwb.base.html#pynwb.base.Images) container in NWB. 

The manual mask and contrast based vasculature mask for the blue channel (full size image) can be accessed 
as `nwbfile.processing["ophys"]["SegmentationImagesBlue"]`.


In [ ]:
images_blue = nwbfile.processing["ophys"]["SegmentationImagesBlue"]
images_blue

In [ ]:
plt.imshow(photon_series_blue.data[50].T, aspect="auto", cmap="Greys")
plt.title("Blue")
plt.show()

plt.imshow(images_blue.images["manual"].data[:].T, aspect="auto", cmap="Greys")
plt.title("Manual mask")
plt.show()

plt.imshow(images_blue.images["vasculature"].data[:].T, aspect="auto", cmap="Greys")
plt.title("Vasculature mask")
plt.show()

The PCA mask and the vasculature mask for the blue channel on the binned session image can be accessed from 
`nwbfile.processing["ophys"]["SegmentationImages"]`.

In [ ]:
nwbfile.processing["ophys"]["SegmentationImages"]

In [ ]:
images_blue = nwbfile.processing["ophys"]["SegmentationImages"]

plt.imshow(nwbfile.processing["ophys"]["OnePhotonSeriesProcessedBlue"].data[50].T, aspect="auto", cmap="Greys")
plt.title("Blue")
plt.show()

plt.imshow(images_blue.images["pca_blue"].data[:].T, aspect="auto", cmap="Greys")
plt.title("PCA mask")
plt.show()

plt.imshow(images_blue.images["vasculature"].data[:].T, aspect="auto", cmap="Greys")
plt.title("Vasculature mask")
plt.show()

The PCA mask for the violet channel on the binned session image can be accessed from 
`nwbfile.processing["ophys"]["SegmentationImagesProcessedViolet"]`

In [ ]:
images_violet = nwbfile.processing["ophys"]["SegmentationImagesProcessedViolet"]
images_violet

In [ ]:
plt.imshow(nwbfile.processing["ophys"]["OnePhotonSeriesProcessedViolet"].data[50].T, aspect="auto", cmap="Greys")
plt.title("Violet")
plt.show()

plt.imshow(images_violet.images["pca_violet"].data[:].T, aspect="auto", cmap="Greys")
plt.title("PCA mask")
plt.show()

# Access Behavior <a name="access-behavior"></a>

This section demonstrates how to access behavioral data from the [pynwb.behavior](https://pynwb.readthedocs.io/en/stable/pynwb.behavior.html#module-pynwb.behavior) module.

The behavior data is stored in the "behavior" processing module, which can be accessed as `nwbfile.processing["behavior"]`.


In [ ]:
nwbfile.processing["behavior"]

## Access Position

[SpatialSeries](https://pynwb.readthedocs.io/en/stable/pynwb.behavior.html#pynwb.behavior.SpatialSeries) is a subclass of [TimeSeries](https://pynwb.readthedocs.io/en/stable/pynwb.base.html#pynwb.base.TimeSeries) that represents data in space, such as the spatial direction, e.g., of gaze or travel, or position of an animal over time.

The x, y (z) position of the animal is stored stored in `SpatialSeries` object inside the [Position](https://pynwb.readthedocs.io/en/stable/pynwb.behavior.html#pynwb.behavior.Position) container.

The Position container can be accessed as `nwbfile.processing["behavior"]["Position"]`.

In [ ]:
nwbfile.processing["behavior"]["Position"]

The x, y, z position of the animal by ViRMEN iteration can be accessed as `nwbfile.processing["behavior"]["Position"]["SpatialSeries"]`.

In [ ]:
position_by_virmen = nwbfile.processing["behavior"]["Position"]["SpatialSeries"]
position_by_virmen

The x, y, position of the animal averaged over the iterations for each frame can be accessed as `nwbfile.processing["behavior"]["Position"]["SpatialSeriesByImFrame"]`.

In [ ]:
position_by_frame = nwbfile.processing["behavior"]["Position"]["SpatialSeriesByImFrame"]
position_by_frame

The "SpatialSeriesByImFrame" has the same time basis as the imaging data (blue frames):

In [ ]:
position_by_frame.timestamps.shape, photon_series_blue.timestamps.shape,

# Access Eye Tracking <a name="access-eyetracking"></a>

This section demonstrates how to access the pose estimation data acquired from Lightning Pose.

The original video is added as [ImageSeries](https://pynwb.readthedocs.io/en/stable/pynwb.image.html#pynwb.image.ImageSeries) with *external* mode. In external mode the video data is not stored in NWB, instead we use [external links](https://www.dandiarchive.org/2022/03/03/external-links-organize.html) to video files using a relative path to that file on disk. In this case the `data` attribute of [ImageSeries](https://pynwb.readthedocs.io/en/stable/pynwb.image.html#pynwb.image.ImageSeries) is empty, instead we have `external_file` attribute:

In [ ]:
nwbfile.acquisition['ImageSeriesOriginalVideo']

In [ ]:
nwbfile.acquisition['ImageSeriesOriginalVideo'].external_file[:][0]

Similarly to the original video, the labeled video (if available) is also added as [ImageSeries](https://pynwb.readthedocs.io/en/stable/pynwb.image.html#pynwb.image.ImageSeries) with *external* mode.

The labeled video is added to the "behavior" processing module and can be accessed as `nwbfile.processing["behavior"]["ImageSeriesLabeledVideo"]`.

In [ ]:
nwbfile.processing["behavior"]["ImageSeriesLabeledVideo"]

To store the pose estimation data in NWB, we are using an NWB extension [ndx-pose](https://github.com/rly/ndx-pose). The `PoseEstimation` container stores the estimated position data (`PoseEstimationSeries`) for multiple body parts computed from the original video.

We can access `PoseEstimation` as `nwbfile.processing["behavior"]["PoseEstimation"]`.

In [ ]:
nwbfile.processing["behavior"]["PoseEstimation"]

The `PoseEstimationSeries` stores the estimated positions (x, y) of a body part over time as well as the confidence of the estimated positions.

In [ ]:
nwbfile.processing["behavior"]["PoseEstimation"].pose_estimation_series

In [ ]:
nwbfile.processing["behavior"]["PoseEstimation"]["PoseEstimationSeriesDRpupil"]

In [ ]:
nwbfile.processing["behavior"]["PoseEstimation"]["PoseEstimationSeriesDRpupil"].data[:10]

In [ ]:
nwbfile.processing["behavior"]["PoseEstimation"]["PoseEstimationSeriesDRpupil"].confidence[:10]

In [ ]:
from nwbwidgets import nwb2widget

nwb2widget(nwbfile)

We also use [Neurosift](https://github.com/flatironinstitute/neurosift), a platform for the visualization of neuroscience data in the web browser.